# Object Tracking with YoloV7 and Pixeltable

In [ ]:
import sys, glob
import pandas as pd
import numpy as np
sys.path.append('/home/marcel/pixeltable')
import PIL

In [ ]:
import pixeltable as pt
from pixeltable import catalog
from pixeltable.type_system import ImageType, StringType, IntType, ArrayType, ColumnType
from pixeltable.functions.pil import draw_boxes
%load_ext autoreload
%autoreload 2

We're loading the function in order to use it for a computed column. Note that we don't need to import the packages we needed to create the model in the first place.

In [ ]:
cl = pt.Client()
functions_db = cl.get_db('functions')
yolov7 = functions_db.load_function('yolov7')

Sanity check

In [ ]:
img_file = '/home/marcel/pixeltable/pixeltable/tests/data/imagenette2-160/n03445777_2563.JPEG'
img = PIL.Image.open(img_file)

print(img.size)
display(img)

In [ ]:
yolov7.eval_fn(img)

In [ ]:
videos = cl.get_db('videos')
data = videos.get_table('data')

data[data.frame, yolov7(data.frame)].show(1)

In [ ]:
videos.drop_table('data2', ignore_errors=True)
t = videos.create_table('data2', [
    catalog.Column('video_file', StringType(), nullable=False),
    catalog.Column('frame', ImageType(), nullable=False, indexed=False),
    catalog.Column('frame_idx', IntType(), nullable=False),
])

In [ ]:
t.add_column(catalog.Column('detections', computed_with=yolov7(t.frame)))

In [ ]:
video_filepaths = glob.glob(f'/home/marcel/pixeltable/**/videos/*.mp4', recursive=True)

df = pd.DataFrame({'video_file': video_filepaths[0:1]})

t.insert_pandas(df, video_column='video_file', frame_column='frame', frame_idx_column='frame_idx', fps=1)

In [ ]:
t[t.detections, draw_boxes(t.frame, t.detections)].show(3)